# 開発用ノートブック1

In [70]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv

必要な機能は
- notionDBからテキストを取得
- テキストをopenAIへ送信、返信を受け取る
- 受け取ったテキストをnotionDBに保存

*大きく分けて二つのサービスをAPIを通じて接続する。まずはローカルで実行できる形にする。*
## notion関連
テキストの取得

In [71]:

# .envファイルから環境変数を読み込む
load_dotenv()

# Notion API
notion_api_key = os.getenv("NOTION_TOKEN")
notion_db_input_id = os.getenv("NOTION_DB_INPUT")

if not notion_api_key or not notion_db_input_id:
    raise ValueError("環境変数 NOTION_TOKEN または NOTION_DB_INPUT が設定されていません")

# config header
headers = {
    "Authorization": f"Bearer {notion_api_key}",
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28"
}

try:
    # データベースをクエリするリクエスト
    response = requests.post(
        f"https://api.notion.com/v1/databases/{notion_db_input_id}/query",
        headers=headers
    )
    
    # レスポンスを取得
    response.raise_for_status()  # エラーチェック
    data = response.json()
    
    # 各ページのIDを取得
    results = data.get('results', [])  
    if not results:
        print("データベースにページが存在しません")
    else:
        for result in results:
            page_id = result.get('id')
            #print(f"Page ID: {page_id}")

except requests.exceptions.RequestException as e:
    print(f"APIリクエストエラー: {e}")
except Exception as e:
    print(f"予期せぬエラー: {e}")

In [72]:
print(result.get('properties').get('タイトル'))

{'id': 'title', 'type': 'title', 'title': [{'type': 'text', 'text': {'content': '進捗レポート', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'plain_text': '進捗レポート', 'href': None}]}


In [73]:
def get_database_items_id(Notion_API_Key, Database_Id, X_Days_Ago=None):


    # ヘッダーを設定
    headers = {
        "Authorization": f"Bearer {Notion_API_Key}",
        "Content-Type": "application/json",
        "Notion-Version": "2022-06-28"
    }

    try:
        # データベースをクエリするリクエスト
        response = requests.post(
            f"https://api.notion.com/v1/databases/{Database_Id}/query",
            headers=headers
        )
        
        # レスポンスを取得
        response.raise_for_status()  # エラーチェック
        data = response.json()
        
        # 各ページの情報を辞書に追加
        results = data.get('results', [])  
        if not results:
            print("データベースにページが存在しません")
            return pd.DataFrame()
        else:
            # 複数のページの情報を格納するためにリストを使用
            pages = []
            
            for result in results:
                
                page_info = {
                    'title': result.get('properties').get('タイトル').get('title')[0].get('plain_text'),
                    'page_id': result.get('id'),
                    'created_time': result.get('created_time')
                }
                pages.append(page_info)

    except requests.exceptions.RequestException as e:
        print(f"APIリクエストエラー: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"予期せぬエラー: {e}")
        return pd.DataFrame()

    # DataFrameを作成
    df = pd.DataFrame(pages)

    # 日付でフィルタリング
    if X_Days_Ago is not None:
        current_date = pd.Timestamp.now(tz='UTC')
        x_days_ago = current_date - pd.Timedelta(days=X_Days_Ago)
        df = df[pd.to_datetime(df['created_time']).between(x_days_ago, current_date)]

    return df

In [74]:
def fetch_blocks(NOTION_API_KEY, page_id: str, page_size: int = 100):
    url = f"https://api.notion.com/v1/blocks/{page_id}/children"
    headers = {
        "Authorization": f"Bearer {NOTION_API_KEY}",
        "Content-Type": "application/json",
        "Notion-Version": "2022-06-28"
    }
    params = {"page_size": page_size}

    blocks = []
    while True:
        resp = requests.get(url, headers=headers, params=params)
        resp.raise_for_status()
        data = resp.json()
        blocks.extend(data["results"])
        if not data.get("has_more"):
            break
        params["start_cursor"] = data["next_cursor"]
    return blocks


In [75]:
def extract_plain_text(block):
    t = ""
    # 多くのブロックは block[block["type"]]["rich_text"] を持つ
    rich = block.get(block["type"], {}).get("rich_text", [])
    for part in rich:
        t += part.get("plain_text", "")
    return t


In [76]:
def page_to_text(page_id):
    blocks = fetch_blocks(notion_api_key, page_id)
    lines = []
    for blk in blocks:
        text = extract_plain_text(blk).strip()
        if text:
            lines.append(text)
        
    return "\n".join(lines)

In [77]:
def get_database_text(NOTION_API_KEY: str, DB_ID: str, X_DAYS_AGO: int = 7):
    page_id_list = get_database_items_id(NOTION_API_KEY, DB_ID, X_DAYS_AGO)['page_id'].to_list()

    text_list = []
    for page_id in page_id_list:
        text_list.append(page_to_text(page_id))
    
    # 各テキストの間に区切り文字を挿入して結合
    combined_text = "<|DOCUMENT|>\n" + "\n<|DOCUMENT|>\n".join(text_list) + "\n<|DOCUMENT|>"
    return combined_text


In [78]:
my_db_id = os.getenv("NOTION_DB_INPUT")

In [79]:
first_system_prompt = r"""
# instruction
You must answer in Japanese
You are professional summarization secretary
You are assigned to quantum information and technology research group
You are not need to be nice and kind
Reference is very important for scientists.

# goal
Generate summary of weekly reports-per-member for professor who unable to attend the meeting. First generate a paragraph of good aspects of group and bad aspect of bad aspect. Second, generate summary of each members. Third, include information as many as possible. If input has url and doi of paper, add full title of paper at the reference section.

# output format
- Output will be passed another generative ai to convert into notion api blocks JSON array

# output template
\# チームの良い点
- 

\# チームの悪い点
- 

\# 各メンバーの進捗
\#\# 名前：
-
\#\# 今週行った事：
paragraph
\#\# 詰まっている箇所：
paragraph or bulleted
\#\# 来週のタスク：
paragraph or bulleted
\#\# 任意記述：
paragraph
\#\# Reference：
-

# input example
<|DOCUMENT|>
作成者：作成者A
今週行った事
K. Toccacelo, U. L. Andersen, and J. B. Brask, Benchmarks for quantum communication via gravity, arXiv: 2503.03585を読む
QIT予稿作成
予稿に向けた図の修正．主にアクセシビリティの向上を目的に．
Qutritエンタングルメントスワッピングの評価を忠実度から対数ネガティビティに置き換える
Journal Club準備．論文: J. Sperling et al., PRA 85 023820 (2012)
課題・詰まっている箇所
特になし
貰ったフィードバック
特になし
来週のタスク
Qutritエンタングルメントスワッピングのプロトコルをquditに一般化
QITの予稿を元に論文を執筆．
自由記述・参考リンク
quantum gravityの論文についてメモ．
重力による近傍にある二つのmassiveな物理系間の通信．
万有引力により，片方の機械振動子の状態がもう一方の機械振動子の状態に移る（ビームスプリッタ相互作用）
このようなswap操作をLOCCで再現しようとした場合のチャンネル忠実度の上限を，実験的に得られた忠実度が上回れば重力場は量子的に取り扱える証左になる．
従来プロトコルでは両方の量子系を計測する必要があったが，今回は片方の量子系をトレースアウトして，残った一方を測定した場合のboundを導出した．
量子性を示すための上限はより厳しいものになったが，ノイズにはロバストになった．
<|DOCUMENT|>
作成者：作成者B
今週行った事
新しくrealistic GKP状態の精度等の見積もりを行った（JSTの申請書に書いた内容）．
phothon subtractionで生成されるsqueezed cat (SC)状態と理想的なSC状態の忠実度は0.969でした．
上記の状態をcat breedingして生成されたGKP状態の忠実度は0.967
理想的なphothon subtractionとcat breedingから生成されるGKP状態はあまり直行しなそうです．
\ket{0_L}と\hat{X_L}\ket{0_L}の忠実度が0.47だった．（理想的なSC状態とcat breedingから生成された場合は0.16）
文献調査「M. Banić, et al., arXiv:2504.10606」．XanaduとUniversity of Calgaryによる最新の研究（14 Apr 2025）．
realisticなGKPクラスタ状態のシミュレーションを行うために，非ガウシアン状態をガウシアン状態の重ね合わせで近似する手法の提案．
平均光子数が大きくなるとFock基底での厳格シミュレーションとの誤差が大きくなる．
文献調査「A. K. Pati, et al., arXiv preprint quant-ph/0002082」．
文献調査「S. Buck, et al., arXiv:2107.02151」．
CV版グローバーアルゴリズムに関する論文．
課題・詰まっている箇所
数値シミュレーションの結果，ロスなしのrealistic GKP状態（phothon subtraction ＋ cat breeding）の直行性が悪いこと．数値シミュレーションのコードが間違っているか，最適なパラメータを選択していないだけだと嬉しい．
貰ったフィードバック
なし
来週のタスク
文献調査「S. Koudia, Phys. Scr. 99, 015115(2024)」
自由記述・参考リンク
測定の射影$\ket{0}\bra{0]$を$e^{-\hat{n}}$のようにdamping$e^{\beta \hat{n}}$と同じようにかける物理的な意味とは？

"""

In [80]:
second_system_prompt = r"""
# instruction
You are markdown converter from markdown to Notion API Blocks array (JSON Array)

# goal
Convert input markdown into Notion API Blocks array (JSON Array)

# output format
- Output always in Notion API blocks array, only JSON array.
- Style the ouput as same as in input

# output template
[
  {
    "object": "block",
    "type": "heading_1",
    "heading_1": {
      "rich_text": [
        {
          "type": "text",
          "text": {
            "content": "ここに見出しレベル1のテキスト"
          }
        }
      ]
    }
  },
  {
    "object": "block",
    "type": "heading_2",
    "heading_2": {
      "rich_text": [
        {
          "type": "text",
          "text": {
            "content": "ここに見出しレベル2のテキスト"
          }
        }
      ]
    }
  },
  {
    "object": "block",
    "type": "paragraph",
    "paragraph": {
      "rich_text": [
        {
          "type": "text",
          "text": {
            "content": "ここに段落テキスト"
          }
        }
      ]
    }
  },
  {
    "object": "block",
    "type": "bulleted_list_item",
    "bulleted_list_item": {
      "rich_text": [
        {
          "type": "text",
          "text": {
            "content": "箇条書きアイテム1"
          }
        }
      ]
    }
  },
  {
    "object": "block",
    "type": "bulleted_list_item",
    "bulleted_list_item": {
      "rich_text": [
        {
          "type": "text",
          "text": {
            "content": "箇条書きアイテム2"
          }
        }
      ]
    }
  },
  {
    "object": "block",
    "type": "numbered_list_item",
    "numbered_list_item": {
      "rich_text": [
        {
          "type": "text",
          "text": {
            "content": "番号付きリストアイテム1"
          }
        }
      ]
    }
  },
  {
    "object": "block",
    "type": "to_do",
    "to_do": {
      "rich_text": [
        {
          "type": "text",
          "text": {
            "content": "チェックリストアイテム"
          }
        }
      ],
      "checked": false
    }
  }
}

# input example
markdown
"""

In [81]:
from openai import OpenAI

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=OPENAI_API_KEY)

def suumarize_text(text):

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": first_system_prompt},
            {"role": "user", "content": f"次の文章を要約し,markdownで出力せよ:\n{text}"}
        ],
        temperature = 0.7
    )

    markdown_response = response.choices[0].message.content

    response = client.chat.completions.create(
        model = "gpt-4o",
        messages=[
            {"role": "system", "content": second_system_prompt},
            {"role": "system", "content": f"Convert markdown input to NOotion API Blocks JSON Array:\n{markdown_response}"}
        ],
        temperature = 0.2
    )
    return response.choices[0].message.content

In [82]:
gen_page = suumarize_text(get_database_text(notion_api_key, my_db_id, 7))

In [83]:
output_db = os.getenv("NOTION_DB_OUTPUT")

In [84]:
import datetime
import json

def post_page(notion_api_key, database_id, blocks_json):

    blocks_json = blocks_json.replace('```json\n', '').replace('\n```', '')

    url = "https://api.notion.com/v1/pages"
    
    headers = {
        "Authorization": f"Bearer {notion_api_key}",
        "Content-Type": "application/json",
        "Notion-Version": "2022-06-28"
    }
    
    data = {
        "parent": {"database_id": database_id},
        "properties": {
            "タイトル": {
                "title": [
                    {
                        "text": {
                            "content": f"AI進捗要約 {datetime.datetime.today().strftime('%Y/%m/%d')}"
                        }
                    }
                ]
            }
        },
        "children": json.loads(blocks_json)
    }
    
    response = requests.post(url, headers=headers, json=data)
    
    if response.status_code == 200:
        print("ページの作成に成功しました")
        #return response.json()
    else:
        print(f"エラーが発生しました: {response.status_code}")
        #print(response.text)
        return None

# 要約したテキストをNotionデータベースに投稿
post_page(notion_api_key, output_db, gen_page)


ページの作成に成功しました


In [85]:
with open("prompt/first_prompt.txt", "r", encoding="utf-8") as f:
        first_system_prompt = f.read()

first_system_prompt

'first_system_prompt = r"""\n# instruction\nYou must answer in Japanese\nYou are professional summarization secretary\nYou are assigned to quantum information and technology research group\nYou are not need to be nice and kind\nReference is very important for scientists.\n\n# goal\nGenerate summary of weekly reports-per-member for professor who unable to attend the meeting. First generate a paragraph of good aspects of group and bad aspect of bad aspect. Second, generate summary of each members. Third, include information as many as possible. If input has url and doi of paper, add full title of paper at the reference section.\n\n# output format\n- Output will be passed another generative ai to convert into notion api blocks JSON array\n\n# output template\n\\# チームの良い点\n- \n\n\\# チームの悪い点\n- \n\n\\# 各メンバーの進捗\n\\#\\# 名前：\n-\n\\#\\# 今週行った事：\nparagraph\n\\#\\# 詰まっている箇所：\nparagraph or bulleted\n\\#\\# 来週のタスク：\nparagraph or bulleted\n\\#\\# 任意記述：\nparagraph\n\\#\\# Reference：\n-\n\n# input ex